# Algorithm Flow Chart

![image](../docs/ETF%20Portfolio%20Momentum%20Strategy%20Flow%20Chart.png)
![image](../docs/ETF%20Portfolio%20Momentum%20Strategy%20Detailed.png)

# Rebalancing Frequency
According to the Monte Carlo optimization, the best rebalancing frequency is 14 days. 
# Percentage of Universe to Trade
According to the Monte Carlo optimization, the best percentage of the universe to trade is 0.38

In [1]:
# Installs (uncomment the line below if any import errors are encountered)

#!pip install numpy pandas yfinance scipy statsmodels ib_insync ipywidgets

In [2]:
# Imports

import os
import math
import time
import logging
import numpy as np
import yfinance as yf
import scipy.stats as sps
import ipywidgets as widgets
import statsmodels.api as sm

from ib_insync import *
from IPython.display import display
from datetime import datetime, timedelta

In [ ]:
# Start connection to IBKR's Trader Workstation (TWS)

HOST = '127.0.0.1'
PORT = 7497
CLIENT_ID = 12

util.startLoop()
ib_client = IB()
ib_client.connect(HOST, PORT, CLIENT_ID)

In [4]:
# Parameters

rsi_window = 14         # Short window for RSI calculation (Default: 14)
hurst_power = 8         # 2^hurst_power is the number of days looked at by the hurst computation (Default: 8)
long_percentile = 6/16  # Percentage of securities actively traded out of the universe (of 16 stocks) (Default: 6/16)
num_stocks = 16         # Total number of stocks in the universe (Default: 16)

In [5]:
# Universe of securities

tickers = [
    
    # Sector ETFs
    'XLY',  # Consumer Discretionary Select Sector SPDR Fund
    'XLP',  # Consumer Staples Select Sector SPDR Fund
    'XLE',  # Energy Select Sector SPDR Fund
    'XLF',  # Financial Select Sector SPDR Fund
    'XLV',  # Health Care Select Sector SPDR Fund
    'XLI',  # Industrial Select Sector SPDR Fund
    'XLB',  # Materials Select Sector SPDR Fund
    'XLRE', # Real Estate Select Sector SPDR Fund
    'XLK',  # Technology Select Sector SPDR Fund
    'XLU',  # Utilities Select Sector SPDR Fund

    # Additional ETFs
    'SCHA', # Schwab U.S. Small-Cap ETF
    'VONG', # Vanguard Russell 1000 Growth ETF
    'IWD',  # iShares Russell 1000 Value ETF
    'IDEV', # iShares International Developed ETF
    'INDA', # iShares MSCI India ETF
    'EWJ',  # iShares MSCI Japan ETF
]

In [6]:
# Features (Hurst exponent, RSI)

def compute_hurst_exponent(data, power):
        n = 2**power
        # Compute returns
        prices = np.array(data)[1:]
        returns = prices / np.array(data)[:-1] - 1
        # Initialize empty arrays
        hursts = np.array([])
        tstats = np.array([])
        pvalues = np.array([])
        # Start sliding the rolling window from n to the end of array
        # (need to wait for n values to compute the first hurst exponent value)
        for t in np.arange(n, len(returns) + 1):
            # Rolling window sample
            data = returns[t-n:t]
            # Generate list of powers of two
            X = np.arange(2, power + 1)
            # Initialize empty array for mean adjusted series
            Y = np.array([])
            # Iterate through list of powers of two
            for p in X:
                # Indexing for window subdivisions
                m = 2**p
                s = 2**(power - p)
                rs_array = np.array([])
                for i in np.arange(0, s):
                    # Subsample window (depends on m which depends on the power p)
                    subsample = data[i*m:(i+1)*m]
                    # Compute mean
                    mean = np.average(subsample)
                    deviate = np.cumsum(subsample - mean)
                    difference = max(deviate) - min(deviate)
                    stdev = np.std(subsample)
                    rescaled_range = difference / stdev
                    rs_array = np.append(rs_array, rescaled_range)
                Y = np.append(Y, np.log2(np.average(rs_array)))
            reg = sm.OLS(Y, sm.add_constant(X))
            res = reg.fit()
            hurst = res.params[1]
            tstat = (res.params[1] - 0.5) / res.bse[1]
            pvalue = 2 * (1 - sps.t.cdf(abs(tstat), res.df_resid))
            hursts = np.append(hursts, hurst)
            tstats = np.append(tstats, tstat)
            pvalues = np.append(pvalues, pvalue)
        
        return hursts, tstats, pvalues

def compute_rsi(data, window=14):
    """Compute the RSI for the given data."""
    delta = data.diff()
    gain = delta.where(delta > 0, 0).rolling(window=window, min_periods=1).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=window, min_periods=1).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return 100 - rsi

In [7]:
# Trading logic

# Configure logging
log_file = 'notebook_logs/trade_log.log'

logging.basicConfig(filename=log_file,
                    filemode='a',
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

orders = {}

def close_position(ticker_symbol):
    # Retrieve current positions
    current_positions = ib_client.positions()
    
    # Find the position for the specified ticker symbol
    for position in current_positions:
        if position.contract.symbol == ticker_symbol:
            # Create a market order to sell the entire position
            stock_contract = position.contract
            shares_to_sell = position.position
            
            if shares_to_sell > 0:
                order = MarketOrder("SELL", shares_to_sell)
                orders[stock_contract] = order
                logging.info(f"Placing order to sell {shares_to_sell} shares of {ticker_symbol}")
                print(f"Placing order to sell {shares_to_sell} shares of {ticker_symbol}")
            else:
                logging.info(f"No shares to sell for {ticker_symbol}")
                print(f"No shares to sell for {ticker_symbol}")
            break
    else:
        logging.error(f"No open position found for {ticker_symbol}")
        print(f"No open position found for {ticker_symbol}")

def order_target_percent(ticker, target_percent):
    # Fetch account summary
    account_summary = ib_client.accountSummary()
    portfolio_value = None

    for item in account_summary:
        if item.tag == 'NetLiquidation' and item.currency == 'USD':
            portfolio_value = float(item.value)
            break

    if portfolio_value is None:
        raise ValueError("Unable to fetch portfolio value.")

    # Fetch the current price of the security
    stock = Stock(ticker, exchange="SMART", currency="USD")
    ib_client.qualifyContracts(stock)
    [ticker] = ib_client.reqTickers(stock)
    stock_price = ticker.marketPrice()

    if stock_price is None or stock_price <= 0:
        raise ValueError("Unable to fetch a valid stock price.")

    logging.info(f"Current price of {ticker.contract.symbol}: ${stock_price:.2f}")
    print(f"Current price of {ticker.contract.symbol}: ${stock_price:.2f}")

    # Calculate the target value and the current value of the position
    target_value = portfolio_value * (target_percent / 100)
    current_position = ib_client.positions()
    current_shares = sum(position.position for position in current_position if position.contract.symbol == ticker.contract.symbol)
    current_value = current_shares * stock_price

    # Calculate the difference and the number of shares to buy/sell
    value_difference = target_value - current_value
    shares_to_trade = int(value_difference / stock_price)

    if abs(shares_to_trade) > 0:
        if shares_to_trade > 0:
            order = MarketOrder("BUY", shares_to_trade)
            action = "buy"
        else:
            order = MarketOrder("SELL", abs(shares_to_trade))
            action = "sell"

        orders[stock] = order
        logging.info(f"Placing order to {action} {abs(shares_to_trade)} shares of {ticker.contract.symbol}")
        print(f"Placing order to {action} {abs(shares_to_trade)} shares of {ticker.contract.symbol}")
    else:
        logging.info(f"No trade necessary for {ticker.contract.symbol}")
        print(f"No trade necessary for {ticker.contract.symbol}")

    logging.info(f"Target allocation: ${target_value:.2f} ({target_percent}% of portfolio)")
    print(f"Target allocation: ${target_value:.2f} ({target_percent}% of portfolio)")
    logging.info(f"Current allocation: ${current_value:.2f} ({(current_value/portfolio_value)*100:.2f}% of portfolio)")
    print(f"Current allocation: ${current_value:.2f} ({(current_value/portfolio_value)*100:.2f}% of portfolio)")

def generate_rebalancing_orders(tickers):
    logging.info('*' * 30)
    logging.info(f"Generating orders for {datetime.now()}")
    start_date = datetime(2023, 1, 1)
    end_date = datetime.now() + timedelta(days=7)

    hurst = 0
    pvalue = 0
    rsi_dict = {}

    spy_df = yf.download("SPY", start_date, end_date, progress=False)
    hursts, tstats, pvalues = compute_hurst_exponent(spy_df['Close'].to_numpy(), hurst_power)
    hurst = hursts[-1]
    pvalue = pvalues[-1]
    logging.info(f"Hurst Exponent: {hurst}, P-value: {pvalue}")
    print(f"Hurst Exponent: {hurst}, P-value: {pvalue}")

    if (hurst > 0.5 and pvalue < 0.05):
        logging.info("Market is trending, applying momentum strategy.")
        print("Market is trending, applying momentum strategy.")
        for ticker in tickers: 
            df = yf.download(ticker, start_date, end_date, progress=False)
            rsi_dict[ticker] = compute_rsi(df['Close'], rsi_window).iloc[-1]

        ranked_stocks = sorted(rsi_dict.items(), key=lambda x: x[1], reverse=True)
        num_to_long = math.floor(long_percentile * num_stocks)
        top_stocks = ranked_stocks[:num_to_long]

        total_rsi = sum(rsi for _, rsi in top_stocks)
        allocation_factor = 0.95  # Allocate 95% of the portfolio

        # Get current positions from IBKR
        current_positions = ib_client.positions()
        current_tickers = [position.contract.symbol for position in current_positions]
        top_stock_symbols = {ticker for ticker, _ in top_stocks}

        logging.info(f"Top stocks: {top_stocks}")
        print(f"Top stocks: {top_stocks}")
        # Close positions that are not in top_stocks
        for position in current_positions:
            if position.contract.symbol not in top_stock_symbols:
                logging.info(f"Closing position for {position.contract.symbol}")
                print(f"Closing position for {position.contract.symbol}")
                close_position(position.contract.symbol)


        for ticker, rsi in top_stocks:
            print('*' * 20)
            # Calculate the proportion of the portfolio to allocate to this stock
            allocation_proportion = (rsi / total_rsi) * allocation_factor
            target_percent = allocation_proportion * 100  # Convert to percentage

            logging.info(f"Allocating {target_percent:.2f}% to {ticker} (RSI: {rsi:.2f})")
            print(f"Allocating {target_percent:.2f}% to {ticker} (RSI: {rsi:.2f})")
            order_target_percent(ticker, target_percent)
            print('*' * 20)


        total_allocated = sum((rsi / total_rsi) * allocation_factor * 100 for _, rsi in top_stocks)
        logging.info(f"Total allocated: {total_allocated:.2f}%")
        print(f"Total allocated: {total_allocated:.2f}%")
        logging.info(f"Cash reserve: {(100 - total_allocated):.2f}%")
        print(f"Cash reserve: {(100 - total_allocated):.2f}%")

    else: 
        logging.info("Market is not trending, exiting strategy.")
        print("Market is not trending, exiting strategy.")
        return

In [ ]:
# Show orders

generate_rebalancing_orders(tickers)

In [ ]:
# Path to the rebalance history file
rebalance_history_file = 'notebook_logs/rebalance_history.txt'

# Function to read the last line of the rebalance history file
def read_last_rebalance_date(filepath):
    try:
        with open(filepath, 'r') as file:
            lines = file.readlines()
            if not lines:
                return "No rebalance history available."
            return lines[-1].strip()  # Return the last line
    except Exception as e:
        return f"Error reading rebalance history file: {e}"

# Create a button widget
confirm_button = widgets.Button(description="Confirm Rebalance", button_style='warning')  # Initial 'warning' state
confirm_state = {'confirmed': False}  # Track the confirmation state

# Define what happens when the button is clicked
def on_button_click(b):
    if not confirm_state['confirmed']:
        # First click - ask for confirmation
        print("This will execute all orders -- are you sure you want to continue?")
        b.description = "Execute Orders"  # Change the button description
        b.button_style = 'danger'  # Change the button color to indicate the seriousness
        confirm_state['confirmed'] = True  # Set the confirmation state
    else:
        # Second click - execute orders
        print("Executing all orders...")
        for contract, order in orders.items():
            trade = ib_client.placeOrder(contract, order)
        
        # Reset button to initial state
        b.description = "Confirm Rebalance"
        b.button_style = 'warning'
        confirm_state['confirmed'] = False  # Reset confirmation state
        
        # Disconnect from the IB client
        ib_client.disconnect()
        print("Orders placed, disconnecting from client")

        # Log the portfolio rebalance date
        logging.info(f"Last portfolio rebalance date: {datetime.now()}")
        try:
            with open(rebalance_history_file, 'a') as file:
                file.write(f"Last portfolio rebalance date: {datetime.now()}\n")
        except Exception as e:
            print(f"Error writing to rebalance history file: {e}")


# Link the button click to the function
confirm_button.on_click(on_button_click)

# Display the button
display(confirm_button)

# Read and display the last rebalance date
last_rebalance_date = read_last_rebalance_date(rebalance_history_file)
print(f"{last_rebalance_date}")
